<a href="https://www.kaggle.com/code/tusharhatwar785/airlines-dataset-analysis-pyspark?scriptVersionId=126165141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=318eb044bfe12601e8b1194118a654ed59f974c9dc90fc4ea2373370ee96ee86
  Stored in directory: /root/.cache/pip/wheels/81/1b/c2/e41f4bddafe5564d11b2414a62dd7f5d75fcf65b2d7b7805c4
Successfully built pyspark


In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/2022-us-airlines-domestic-departure-data/Carriers.csv
/kaggle/input/2022-us-airlines-domestic-departure-data/CompleteData.csv
/kaggle/input/2022-us-airlines-domestic-departure-data/ActiveWeather.csv
/kaggle/input/2022-us-airlines-domestic-departure-data/Stations.csv
/kaggle/input/2022-us-airlines-domestic-departure-data/Cancellation.csv


In [3]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print('Spark Version: {}'.format(spark.version))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 16:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.4.0


/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
#Defining a Schema
csvSchema = StructType([StructField('rank', IntegerType(), nullable = True),
                     StructField('title', StringType(), nullable = True),
                     StructField('genre', StringType(), nullable = True),
                     StructField('description', StringType(), nullable = True),
                     StructField('director', StringType(), nullable = True),
                     StructField('actors', StringType(), nullable = True),
                     StructField('year', IntegerType(), nullable = True),
                     StructField('Runtime (Minutes)', IntegerType(), nullable = True),
                     StructField('rating', FloatType(), nullable = True),
                     StructField('votes', IntegerType(), nullable = True),
                     StructField('Revenue (Millions)', FloatType(), nullable = True),
                     StructField('metascore', FloatType(), nullable = True)
                    ])

#Defining a Schema
carrierCsvSchema = StructType([StructField('code', StringType(), nullable = True),
                     StructField('description', StringType(), nullable = True)
                    ])

carrier_file_path = '/kaggle/input/2022-us-airlines-domestic-departure-data/Carriers.csv'


df = spark.read.csv(carrier_file_path,
                    header = True,
                    schema = carrierCsvSchema,
                    nanValue = '?')
df.show(5, False)

+----+-----------------------------+
|code|description                  |
+----+-----------------------------+
|9E  |Endeavor Air Inc.            |
|AA  |American Airlines Inc.       |
|AS  |Alaska Airlines Inc.         |
|B6  |JetBlue Airways              |
|C5  |CommuteAir LLC dba CommuteAir|
+----+-----------------------------+
only showing top 5 rows

